In [1]:
from pulp import *

In [2]:
import init
import SBMLLint.common.constants as cn
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
TOLERANCE = 0.0001

from numpy.linalg import svd
total_time_begin = time.time()

In [3]:
# MODEL = "BIOMD0000000167.xml"
# simple = SimpleSBML()
# simple.initialize(os.path.join(cn.BIOMODELS_DIR, MODEL))

In [4]:
# SAMPLE_MODEL = "iCN718.xml"
SAMPLE_MODEL = "Recon3D.xml"
bigg_filepath = os.path.join(cn.BIGG_DIR, SAMPLE_MODEL)
simple = SimpleSBML()
simple.initialize(bigg_filepath)

In [5]:
simple.reactions[:10]

[R_24_25DHVITD3tm: M_2425dhvitd3_m -> M_2425dhvitd3_c,
 R_25HVITD3t: M_25hvitd3_c -> M_25hvitd3_e,
 R_COAtl: M_coa_c -> M_coa_l,
 R_EX_5adtststerone_e: M_5adtststerone_e -> ,
 R_EX_5adtststerones_e: M_5adtststerones_e -> ,
 R_EX_5fthf_e: M_5fthf_e -> ,
 R_EX_5htrp_e: M_5htrp_e -> ,
 R_EX_5mthf_e: M_5mthf_e -> ,
 R_EX_5thf_e: M_5thf_e -> ,
 R_EX_6dhf_e: M_6dhf_e -> ]

In [6]:
s = StoichiometryMatrix(simple)
mat = s.stoichiometry_matrix
mat.shape

(5835, 8794)

In [7]:
def solveMILP(mat):
  prob = LpProblem("Finding_Unconserved_Metabolites", LpMaximize)
  species = list(mat.index)
  species_inclusion = pulp.LpVariable.dicts("species", species, cat="Binary")
  species_mass = pulp.LpVariable.dicts("mass", species, cat="Continuous")
  # objective function (to maximize the number of species)
  prob += lpSum([species_inclusion[i] for i in species])
  # constraint 1 (for each reaction, the sum(stoichiometry[i]*mass[i])=0)
  for reaction in mat.columns:
    prob += lpSum(sum([mat[reaction][species]*species_mass[species] for species in species_inclusion])) == 0
  # constraint 2 (species_inclusion is less than or equal to mass of each species)
  for species in species_mass.keys():
    prob += species_inclusion[species] <= species_mass[species]
  prob.solve()
  return prob
def getUnconservedMetabolites(milp_result):
  unconserved_metabolites = []
  for v in milp_result.variables():
    if v.varValue==0:
      if v.name[:7]=="species":
        unconserved_metabolites.append(v.name[8:])
  return unconserved_metabolites

In [21]:
# re-run mode / Biomodel or Recon3D
milp_start = time.time()
s = StoichiometryMatrix(simple)
milp_result = solveMILP(s.stoichiometry_matrix)
unconserved_metabolites = getUnconservedMetabolites(milp_result)
milp_end = time.time()
milp_time = milp_end - milp_start
print("MILP time: %f" % milp_time)

KeyboardInterrupt: 

In [9]:
unconserved_metabolites[:5]

['M_11docrtsl_c',
 'M_11docrtsl_e',
 'M_11docrtsl_m',
 'M_11docrtsl_r',
 'M_11docrtstrn_c']

In [10]:
# from i718 model
# R_SEAHCYSHYD: M_h2o_c + M_seahcys_c -> M_adn_c + M_selhcys_c
# R_SEAHCYSHYD_1: M_h2o_c + M_seahcys_c -> M_adn_c + M_h_c + M_selhcys_c
# "M_h_c" in res

In [11]:
# get left nullspace matrix
matrix = np.atleast_2d(mat.T)
_, sigma, vh = svd(matrix)
atol = 1e-13
rtol = 0.0
tol = max(atol, rtol*sigma[0])
num_nonzero = (sigma >= tol).sum()
print(num_nonzero)
left_ns = pd.DataFrame(vh[num_nonzero:].conj().T, index=mat.index)

5600


In [12]:
# list(left_ns.index)

In [13]:
# Finding minimal net inconsistent stoichiometry. 
# target_met is for each unconserved metabolite
min_net_stoi_time_begin = time.time()
target_met = unconserved_metabolites[0]
prob_min_net_stoi = LpProblem("Finding_Minimal_Net_Stoichiometry", LpMinimize)
species = list(left_ns.index)
species_inclusion_net_stoi = pulp.LpVariable.dicts("include", species, cat="Binary")
species_weight_net_stoi = pulp.LpVariable.dicts("weight", species, lowBound=0, cat="Continuous")
# objective function to minimize
prob_min_net_stoi += lpSum([species_inclusion_net_stoi[i] for i in species])
# constraint 1 (for each metabolite, species_weight_net_stoi[i]*species_inclusion_net_stoi[i])=0)
# for metabolite in species:
for col_idx in left_ns.columns:
  left_ns_col = left_ns[col_idx]
  prob_min_net_stoi += lpSum(species_weight_net_stoi[metabolite]*left_ns_col.T[metabolite] for metabolite in species)==0
# constraint 2 (species_inclusion is less than or equal to mass of each species)
for species in species_weight_net_stoi.keys():
  prob_min_net_stoi += species_weight_net_stoi[species] <= species_inclusion_net_stoi[species]
# constraint 3 designated metabolite nonzero
prob_min_net_stoi += species_weight_net_stoi[target_met] >= TOLERANCE
# solve problem
prob_min_net_stoi.solve()
print("Status:", LpStatus[prob_min_net_stoi.status])
included_met = []
for v in prob_min_net_stoi.variables():
  if (v.name[:3]=="inc") & (v.varValue==1):
    included_met.append(v.name[8:])
print(included_met)
min_net_stoi_time_end = time.time()

Status: Optimal
['M_11docrtsl_c', 'M_4mptnl_c', 'M_s2l2n2m2m_l']
Min Net Stoichiometry MILP time: 1451.077650


In [19]:
min_net_stoi_time = min_net_stoi_time_end - min_net_stoi_time_begin 
print("Min Net Stoichiometry MILP time: %f" % min_net_stoi_time)

Min Net Stoichiometry MILP time: 145.603002


In [20]:
145 * 4000

580000

In [14]:
# prob_min_net_stoi

In [15]:
total_time_end = time.time()
print("total time:")
print(total_time_end - total_time_begin)

total time:
1692.6930630207062


In [16]:
# nonzero_loc = mat.T["M_10fthf_c"].to_numpy().nonzero()
# mat.T["M_10fthf_c"].iloc[nonzero_loc]

In [17]:
for v in prob_min_net_stoi.variables():
  if v.varValue != 0.0:
    print(v.name)
    print(v.varValue)

include_M_11docrtsl_c
1.0
include_M_4mptnl_c
1.0
include_M_s2l2n2m2m_l
1.0
weight_M_11docrtsl_c
0.0001
weight_M_4mptnl_c
0.0001
weight_M_s2l2n2m2m_l
1.5071977e-05


In [18]:
prob_min_net_stoi.status

1